# MolecuGen: End-to-End Tutorial

This notebook provides a comprehensive tutorial for using MolecuGen to train molecular generation models and generate drug-like molecules.

## Table of Contents
1. [Setup and Installation](#setup)
2. [Data Preparation](#data-preparation)
3. [Model Training](#model-training)
4. [Molecular Generation](#molecular-generation)
5. [Constraint-Based Generation](#constraint-generation)
6. [Evaluation and Analysis](#evaluation)
7. [Visualization](#visualization)
8. [Performance Benchmarking](#benchmarking)

## 1. Setup and Installation {#setup}

First, let's install the required dependencies and import necessary modules.

In [ ]:
# Install dependencies (run once)
# !pip install torch torch-geometric rdkit-pypi numpy scipy matplotlib seaborn pandas

import sys
import os
sys.path.append('..')  # Add parent directory to path

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

In [ ]:
# Import MolecuGen modules
from src.data.molecular_dataset import MolecularDataset
from src.data.smiles_processor import SMILESProcessor
from src.data.feature_extractor import FeatureExtractor
from src.training.trainer import Trainer
from src.generate.molecular_generator import MolecularGenerator
from src.generate.constraint_filter import ConstraintFilter
from src.evaluate.molecular_evaluator import MolecularEvaluator

print("MolecuGen modules imported successfully!")

## 2. Data Preparation {#data-preparation}

Let's prepare a dataset of molecules for training. We'll use a subset of ZINC15 molecules.

In [ ]:
# Sample SMILES data (in practice, load from file)
sample_smiles = [
    "CCO",  # Ethanol
    "CC(=O)O",  # Acetic acid
    "CC(=O)Nc1ccc(O)cc1",  # Acetaminophen
    "CC(C)Cc1ccc(C(C)C(=O)O)cc1",  # Ibuprofen
    "CN1C=NC2=C1C(=O)N(C(=O)N2C)C",  # Caffeine
    "CC1=CC=C(C=C1)C(=O)O",  # p-Toluic acid
    "C1=CC=C(C=C1)C(=O)O",  # Benzoic acid
    "CC(C)(C)NCC(C1=CC(=C(C=C1)O)CO)O",  # Salbutamol
    "CC1=C(C(=O)N(N1C)C2=CC=CC=C2)C(=O)NCCCCOC3=CC=CC=C3",  # Example drug-like molecule
    "COC1=C(C=CC(=C1)CC2COC(=O)C2)O"  # Another drug-like molecule
]

# In practice, load from file:
# with open('data/zinc15_subset.smi', 'r') as f:
#     sample_smiles = [line.strip() for line in f if line.strip()]

print(f"Loaded {len(sample_smiles)} sample molecules")
for i, smiles in enumerate(sample_smiles[:5]):
    print(f"{i+1}: {smiles}")

In [ ]:
# Initialize data processors
smiles_processor = SMILESProcessor(
    add_self_loops=False,
    explicit_hydrogens=False,
    sanitize=True
)

feature_extractor = FeatureExtractor(
    use_chirality=True,
    use_partial_charge=False,  # Faster without partial charges
    max_atomic_num=100
)

print(f"Atom feature dimensions: {feature_extractor.get_atom_features_dim()}")
print(f"Bond feature dimensions: {feature_extractor.get_bond_features_dim()}")

In [ ]:
# Create molecular dataset
dataset = MolecularDataset(
    smiles_list=sample_smiles,
    smiles_processor=smiles_processor,
    feature_extractor=feature_extractor,
    max_nodes=50
)

print(f"Created dataset with {len(dataset)} valid molecules")

# Examine a sample molecule
sample_data = dataset[0]
print(f"\nSample molecule:")
print(f"SMILES: {sample_data.smiles}")
print(f"Nodes: {sample_data.x.shape[0]}")
print(f"Edges: {sample_data.edge_index.shape[1]}")
print(f"Node features shape: {sample_data.x.shape}")
print(f"Edge features shape: {sample_data.edge_attr.shape}")

## 3. Model Training {#model-training}

Now let's train a small diffusion model on our dataset.

In [ ]:
# Training configuration
config = {
    'name': 'tutorial_model',
    'output_dir': 'tutorial_experiments',
    
    'model': {
        'type': 'diffusion',
        'hidden_dim': 128,  # Small model for tutorial
        'num_layers': 3,
        'dropout': 0.1,
        'num_timesteps': 100,  # Fewer timesteps for faster training
        'beta_schedule': 'cosine',
        'max_nodes': 50
    },
    
    'training': {
        'batch_size': 4,  # Small batch for small dataset
        'num_epochs': 20,  # Few epochs for tutorial
        'learning_rate': 1e-3,
        'weight_decay': 1e-5,
        'gradient_clip': 1.0,
        'patience': 10,
        'save_every': 5,
        'validate_every': 1,
        
        'optimizer': {
            'type': 'adam',
            'betas': [0.9, 0.999]
        },
        
        'scheduler': {
            'type': 'cosine',
            'eta_min': 1e-6
        }
    },
    
    'data': {
        'train_split': 0.7,
        'val_split': 0.2,
        'test_split': 0.1
    },
    
    'logging': {
        'level': 'INFO',
        'use_tensorboard': False,  # Disable for tutorial
        'use_wandb': False
    }
}

print("Training configuration created")

In [ ]:
# Initialize trainer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

trainer = Trainer(config=config, device=device)

# Setup training components
print("Setting up data...")
trainer.setup_data(dataset)

print("Setting up model...")
trainer.setup_model()

print("Setting up optimizer...")
trainer.setup_optimizer()

print("Setup completed!")

In [ ]:
# Train the model
print("Starting training...")
results = trainer.train()

print(f"\nTraining completed!")
print(f"Best validation loss: {results['best_val_loss']:.6f}")
print(f"Total epochs: {results['total_epochs']}")
print(f"Training time: {results['total_time']:.2f} seconds")

In [ ]:
# Plot training progress
history = trainer.training_history

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Plot losses
axes[0].plot(history['epochs'], history['train_losses'], label='Train Loss', marker='o')
axes[0].plot(history['epochs'], history['val_losses'], label='Validation Loss', marker='s')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].set_title('Training Progress')
axes[0].grid(True, alpha=0.3)

# Plot learning rate
axes[1].plot(history['epochs'], history['learning_rates'], color='red', marker='d')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Learning Rate')
axes[1].set_title('Learning Rate Schedule')
axes[1].grid(True, alpha=0.3)
axes[1].set_yscale('log')

plt.tight_layout()
plt.show()

## 4. Molecular Generation {#molecular-generation}

Now let's use our trained model to generate new molecules.

In [ ]:
# Load the trained model
checkpoint_path = Path(config['output_dir']) / "checkpoints" / "best_model.pt"

if checkpoint_path.exists():
    generator = MolecularGenerator.from_checkpoint(checkpoint_path, device=device)
    print("Model loaded successfully!")
else:
    print(f"Checkpoint not found at {checkpoint_path}")
    # Use the current model from trainer
    generator = MolecularGenerator(
        model=trainer.model,
        smiles_processor=smiles_processor,
        device=device
    )
    print("Using current model from trainer")

In [ ]:
# Generate molecules with different temperatures
temperatures = [0.5, 1.0, 1.5, 2.0]
generation_results = {}

for temp in temperatures:
    print(f"\nGenerating with temperature {temp}...")
    molecules = generator.generate(
        num_molecules=20,
        temperature=temp,
        batch_size=5,
        max_attempts=5
    )
    
    generation_results[temp] = molecules
    print(f"Generated {len(molecules)} molecules")
    
    # Show first few molecules
    for i, smiles in enumerate(molecules[:3]):
        print(f"  {i+1}: {smiles}")

In [ ]:
# Analyze generation statistics
stats = generator.get_generation_statistics()
print("Generation Statistics:")
print(f"Total generated: {stats['total_generated']}")
print(f"Valid molecules: {stats['valid_molecules']}")
print(f"Unique molecules: {stats['unique_molecules']}")
print(f"Validity rate: {stats['validity_rate']:.2%}")
print(f"Uniqueness rate: {stats['uniqueness_rate']:.2%}")

## 5. Constraint-Based Generation {#constraint-generation}

Let's generate molecules that satisfy drug-likeness constraints.

In [ ]:
# Define drug-likeness constraints
drug_constraints = {
    'lipinski': True,           # Lipinski's Rule of Five
    'qed_threshold': 0.3,       # Minimum drug-likeness score (lowered for small dataset)
    'mw_range': [100, 500],     # Molecular weight range
    'logp_range': [-2, 5]       # LogP range
}

print("Drug-likeness constraints:")
for key, value in drug_constraints.items():
    print(f"  {key}: {value}")

In [ ]:
# Generate drug-like molecules
print("Generating drug-like molecules...")
drug_molecules = generator.generate_with_constraints(
    num_molecules=15,
    constraints=drug_constraints,
    temperature=1.2,
    max_attempts=10,
    iterative_filtering=True
)

print(f"\nGenerated {len(drug_molecules)} drug-like molecules:")
for i, smiles in enumerate(drug_molecules):
    print(f"{i+1:2d}: {smiles}")

In [ ]:
# Analyze constraint satisfaction
constraint_filter = ConstraintFilter()

print("\nConstraint Analysis:")
for i, smiles in enumerate(drug_molecules[:5]):  # Analyze first 5
    print(f"\nMolecule {i+1}: {smiles}")
    
    # Check individual Lipinski rules
    lipinski_results = constraint_filter.check_lipinski_rule(smiles)
    print(f"  Lipinski rules: MW={lipinski_results['mw_pass']}, "
          f"LogP={lipinski_results['logp_pass']}, "
          f"HBD={lipinski_results['hbd_pass']}, "
          f"HBA={lipinski_results['hba_pass']}")
    
    # Calculate QED score
    qed_score = constraint_filter.calculate_qed_score(smiles)
    print(f"  QED score: {qed_score:.3f}" if qed_score else "  QED score: N/A")
    
    # Calculate molecular weight and LogP
    mw = constraint_filter.calculate_molecular_weight(smiles)
    logp = constraint_filter.calculate_logp(smiles)
    print(f"  MW: {mw:.1f}, LogP: {logp:.2f}" if mw and logp else "  Properties: N/A")

## 6. Evaluation and Analysis {#evaluation}

Let's comprehensively evaluate our generated molecules.

In [ ]:
# Combine all generated molecules for evaluation
all_generated = []
for temp_molecules in generation_results.values():
    all_generated.extend(temp_molecules)
all_generated.extend(drug_molecules)

# Remove duplicates
unique_generated = list(set(all_generated))
print(f"Total unique generated molecules: {len(unique_generated)}")

In [ ]:
# Create evaluator with training data as reference
reference_molecules = sample_smiles  # Use original training data as reference
evaluator = MolecularEvaluator(reference_molecules=reference_molecules)

# Comprehensive evaluation
evaluation_results = evaluator.evaluate(unique_generated)

print("Basic Evaluation Metrics:")
print(f"Validity: {evaluation_results['validity']:.2%}")
print(f"Uniqueness: {evaluation_results['uniqueness']:.2%}")
print(f"Novelty: {evaluation_results['novelty']:.2%}")
print(f"Total molecules: {evaluation_results['total_molecules']}")

In [ ]:
# Drug-likeness evaluation
drug_likeness_metrics = evaluator.compute_drug_likeness_metrics(unique_generated)

print("\nDrug-likeness Metrics:")
print(f"Mean QED: {drug_likeness_metrics['mean_qed']:.3f}")
print(f"Median QED: {drug_likeness_metrics['median_qed']:.3f}")
print(f"Lipinski pass rate: {drug_likeness_metrics['lipinski_pass_rate']:.2%}")
print(f"MW pass rate: {drug_likeness_metrics['mw_pass_rate']:.2%}")
print(f"LogP pass rate: {drug_likeness_metrics['logp_pass_rate']:.2%}")
print(f"HBD pass rate: {drug_likeness_metrics['hbd_pass_rate']:.2%}")
print(f"HBA pass rate: {drug_likeness_metrics['hba_pass_rate']:.2%}")

In [ ]:
# Property distributions
property_distributions = evaluator.compute_property_distributions(unique_generated)
reference_distributions = evaluator.compute_property_distributions(reference_molecules)

print("\nProperty Distribution Statistics:")
properties = ['molecular_weight', 'logp', 'num_atoms', 'num_bonds']

for prop in properties:
    if prop in property_distributions and len(property_distributions[prop]) > 0:
        gen_mean = np.mean(property_distributions[prop])
        gen_std = np.std(property_distributions[prop])
        
        if prop in reference_distributions and len(reference_distributions[prop]) > 0:
            ref_mean = np.mean(reference_distributions[prop])
            ref_std = np.std(reference_distributions[prop])
            print(f"{prop}:")
            print(f"  Generated: {gen_mean:.2f} ± {gen_std:.2f}")
            print(f"  Reference: {ref_mean:.2f} ± {ref_std:.2f}")
        else:
            print(f"{prop}: {gen_mean:.2f} ± {gen_std:.2f}")

## 7. Visualization {#visualization}

Let's create visualizations to better understand our results.

In [ ]:
# Plot property distributions
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

properties = ['molecular_weight', 'logp', 'num_atoms', 'num_bonds']
property_labels = ['Molecular Weight (Da)', 'LogP', 'Number of Atoms', 'Number of Bonds']

for i, (prop, label) in enumerate(zip(properties, property_labels)):
    ax = axes[i]
    
    if prop in property_distributions and len(property_distributions[prop]) > 0:
        # Plot generated molecules
        ax.hist(property_distributions[prop], bins=10, alpha=0.7, 
                label='Generated', color='skyblue', density=True)
        
        # Plot reference molecules if available
        if prop in reference_distributions and len(reference_distributions[prop]) > 0:
            ax.hist(reference_distributions[prop], bins=10, alpha=0.7, 
                    label='Reference', color='orange', density=True)
        
        ax.set_xlabel(label)
        ax.set_ylabel('Density')
        ax.set_title(f'Distribution of {label}')
        ax.legend()
        ax.grid(True, alpha=0.3)
    else:
        ax.text(0.5, 0.5, f'No data for\n{label}', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title(f'{label} (No Data)')

plt.tight_layout()
plt.show()

In [ ]:
# QED score distribution
qed_scores = evaluator.compute_qed_scores(unique_generated)
valid_qed_scores = [score for score in qed_scores if score > 0]

if valid_qed_scores:
    plt.figure(figsize=(8, 5))
    plt.hist(valid_qed_scores, bins=15, alpha=0.7, color='lightgreen', edgecolor='black')
    plt.axvline(np.mean(valid_qed_scores), color='red', linestyle='--', 
                label=f'Mean: {np.mean(valid_qed_scores):.3f}')
    plt.axvline(0.5, color='orange', linestyle='--', 
                label='Drug-like threshold (0.5)')
    plt.xlabel('QED Score')
    plt.ylabel('Frequency')
    plt.title('Distribution of QED Scores (Drug-likeness)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("No valid QED scores to plot")

In [ ]:
# Lipinski compliance visualization
lipinski_compliance = evaluator.compute_lipinski_compliance(unique_generated)

if lipinski_compliance['lipinski_pass']:
    # Calculate pass rates for each rule
    rules = ['molecular_weight_ok', 'logp_ok', 'hbd_ok', 'hba_ok', 'lipinski_pass']
    rule_labels = ['MW ≤ 500', 'LogP ≤ 5', 'HBD ≤ 5', 'HBA ≤ 10', 'All Rules']
    pass_rates = [np.mean(lipinski_compliance[rule]) for rule in rules]
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(rule_labels, pass_rates, color=['lightblue', 'lightgreen', 'lightcoral', 'lightyellow', 'lightpink'])
    plt.ylabel('Pass Rate')
    plt.title('Lipinski Rule Compliance')
    plt.ylim(0, 1.1)
    
    # Add percentage labels on bars
    for bar, rate in zip(bars, pass_rates):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                f'{rate:.1%}', ha='center', va='bottom')
    
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()
else:
    print("No Lipinski compliance data to plot")

## 8. Performance Benchmarking {#benchmarking}

Let's benchmark the generation performance and compare different settings.

In [ ]:
import time

# Benchmark generation speed
def benchmark_generation(generator, num_molecules, batch_sizes, temperatures):
    results = []
    
    for batch_size in batch_sizes:
        for temp in temperatures:
            print(f"Testing batch_size={batch_size}, temperature={temp}")
            
            start_time = time.time()
            molecules = generator.generate(
                num_molecules=num_molecules,
                batch_size=batch_size,
                temperature=temp,
                max_attempts=3
            )
            end_time = time.time()
            
            # Validate molecules
            valid_count = sum(1 for mol in molecules if generator.smiles_processor.validate_molecule(mol))
            
            results.append({
                'batch_size': batch_size,
                'temperature': temp,
                'time': end_time - start_time,
                'generated': len(molecules),
                'valid': valid_count,
                'validity_rate': valid_count / len(molecules) if molecules else 0,
                'molecules_per_second': len(molecules) / (end_time - start_time)
            })
    
    return results

# Run benchmark
benchmark_results = benchmark_generation(
    generator=generator,
    num_molecules=20,
    batch_sizes=[2, 4, 8],
    temperatures=[0.8, 1.0, 1.2]
)

# Convert to DataFrame for easier analysis
benchmark_df = pd.DataFrame(benchmark_results)
print("\nBenchmark Results:")
print(benchmark_df.round(3))

In [ ]:
# Visualize benchmark results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Generation speed vs batch size
for temp in benchmark_df['temperature'].unique():
    temp_data = benchmark_df[benchmark_df['temperature'] == temp]
    axes[0].plot(temp_data['batch_size'], temp_data['molecules_per_second'], 
                marker='o', label=f'T={temp}')

axes[0].set_xlabel('Batch Size')
axes[0].set_ylabel('Molecules per Second')
axes[0].set_title('Generation Speed vs Batch Size')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Validity rate vs temperature
for batch_size in benchmark_df['batch_size'].unique():
    batch_data = benchmark_df[benchmark_df['batch_size'] == batch_size]
    axes[1].plot(batch_data['temperature'], batch_data['validity_rate'], 
                marker='s', label=f'Batch={batch_size}')

axes[1].set_xlabel('Temperature')
axes[1].set_ylabel('Validity Rate')
axes[1].set_title('Validity Rate vs Temperature')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Memory usage analysis (if CUDA is available)
if torch.cuda.is_available():
    print("GPU Memory Usage Analysis:")
    
    # Clear cache
    torch.cuda.empty_cache()
    
    # Measure memory before generation
    memory_before = torch.cuda.memory_allocated() / 1024**2  # MB
    
    # Generate molecules
    test_molecules = generator.generate(num_molecules=10, batch_size=5)
    
    # Measure memory after generation
    memory_after = torch.cuda.memory_allocated() / 1024**2  # MB
    memory_peak = torch.cuda.max_memory_allocated() / 1024**2  # MB
    
    print(f"Memory before generation: {memory_before:.1f} MB")
    print(f"Memory after generation: {memory_after:.1f} MB")
    print(f"Peak memory usage: {memory_peak:.1f} MB")
    print(f"Memory increase: {memory_after - memory_before:.1f} MB")
    
    # Reset peak memory stats
    torch.cuda.reset_peak_memory_stats()
else:
    print("CUDA not available - skipping GPU memory analysis")

## Summary and Next Steps

In this tutorial, we've covered:

1. **Data Preparation**: Loading and processing molecular data
2. **Model Training**: Training a diffusion model for molecular generation
3. **Generation**: Generating molecules with different parameters
4. **Constraint-based Generation**: Generating drug-like molecules
5. **Evaluation**: Comprehensive evaluation of generated molecules
6. **Visualization**: Creating plots to understand results
7. **Benchmarking**: Performance analysis and optimization

### Key Takeaways:
- Temperature affects the diversity vs quality trade-off
- Constraint-based generation helps produce drug-like molecules
- Proper evaluation is crucial for understanding model performance
- Batch size affects generation speed and memory usage

### Next Steps:
1. **Scale up**: Train on larger datasets (ZINC15, ChEMBL)
2. **Advanced constraints**: Implement custom constraint functions
3. **Property prediction**: Integrate property prediction models
4. **Model comparison**: Compare different architectures (GraphAF vs GraphDiffusion)
5. **Production deployment**: Set up model serving for applications

### Resources:
- [Advanced Generation Examples](advanced_generation.md)
- [Production Deployment Guide](production_deployment.md)
- [API Documentation](../docs/api/README.md)
- [Best Practices](../docs/best_practices.md)

In [ ]:
# Save results for future use
results_summary = {
    'training_results': results,
    'evaluation_metrics': evaluation_results,
    'drug_likeness_metrics': drug_likeness_metrics,
    'benchmark_results': benchmark_df.to_dict('records'),
    'generated_molecules': unique_generated,
    'drug_molecules': drug_molecules
}

# Save to file (optional)
import json
output_file = 'tutorial_results.json'

# Convert numpy arrays to lists for JSON serialization
def convert_numpy(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    return obj

# Clean results for JSON serialization
clean_results = json.loads(json.dumps(results_summary, default=convert_numpy))

with open(output_file, 'w') as f:
    json.dump(clean_results, f, indent=2)

print(f"Results saved to {output_file}")
print("Tutorial completed successfully!")